# **Homework 1: Linear Regression**

本次目標：由前 9 個小時的 18 個 features (包含 PM2.5)預測的 10 個小時的 PM2.5。<!-- 可以參考 <link> 獲知更細項的作業說明。-->

<!-- 首先，從 https://drive.google.com/open?id=1El0zvTkrSuqCTDcMpijXpADvJzZC2Jpa 將整個資料夾下載下來，並將下載下來的資料夾放到自己的 Google Drive（注意：上傳到自己 Google Drive 的是資料夾 hw1-regression，而非壓縮檔） -->

# **Load 'train.csv'**
train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [7]:
import sys
import os
import pandas as pd
import numpy as np
from google.colab import drive 
#!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
#!unzip data.zip
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/hw1-regression"
os.chdir(path)
os.listdir(path)

#data = pd.read_csv('./hw1_Regression/DataSet/train.csv', encoding = 'big5')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


['train.csv', 'test.csv', 'hw1_regression.ipynb']

In [8]:
# 读入train.csv，繁体字以big5编码
data = pd.read_csv('train.csv', header = None, encoding = 'big5')

# **Preprocessing** 
取需要的數值部分，將 'RAINFALL' 欄位全部補 0。
另外，如果要在 colab 重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。

In [9]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

# **Extract Features (1)**
![圖片說明](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![圖片說明](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)

將原始 4320 * 18 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。 

In [10]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

# **Extract Features (2)**
![alt text](https://drive.google.com/uc?id=1wKoPuaRHoX682LMiBgIoOP4PDyNKsJLK)
![alt text](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

In [11]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            # 最后一天的最后9个小时不能算进去，不然会越界
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

[[0.  1.  2.  ... 2.5 2.  0.6]
 [1.  2.  3.  ... 2.  0.6 0.8]
 [2.  3.  4.  ... 0.6 0.8 1.6]
 ...
 [0.8 1.8 2.6 ... 1.5 1.  1.7]
 [1.8 2.6 1.4 ... 1.  1.7 1.5]
 [2.6 1.4 1.7 ... 1.7 1.5 2. ]]
[[38.]
 [56.]
 [64.]
 ...
 [50.]
 [32.]
 [22.]]


# **Normalize (1)**
$x = \frac{x - \mu}{\sigma}$, 其中，$\mu$是样本的均值，$\sigma$是样本的标准差。

通过标准化，可以：

- 将有量纲的表达式，经过变换，化为无量纲的表达式，成为标量
- 使得数据更加符合独立同分布条件

In [12]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

array([[-1.32940065, -0.57052916,  0.18849156, ...,  0.18190364,
        -0.28611588, -1.59862324],
       [-0.57043373,  0.18838607,  0.9474531 , ..., -0.28631722,
        -1.59712273, -1.41122474],
       [ 0.18853318,  0.9473013 ,  1.70641464, ..., -1.59733562,
        -1.40983604, -0.66163075],
       ...,
       [-0.72222712,  0.03660302,  0.64386849, ..., -0.75453808,
        -1.22254934, -0.5679315 ],
       [ 0.0367398 ,  0.64373521, -0.26688536, ..., -1.22275893,
        -0.56704592, -0.75533   ],
       [ 0.64391333, -0.26696307, -0.0391969 , ..., -0.56724973,
        -0.75433261, -0.28683376]])

## Shuffling Data

In [13]:
per = np.random.permutation(x.shape[0])  #打乱后的行号
x = x[per, :]
y = y[per]
print(x.shape)
print(y.shape)

(5652, 162)
(5652, 1)


#**Split Training Data Into "train_set" and "validation_set"**
這部分是針對作業中 report 的第二題、第三題做的簡單示範，以生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[ 0.23285597  0.23170375  0.23055081 ... -0.29578943 -0.29644471
   0.07797893]
 [-0.7218096  -0.72262212 -0.72331287 ... -1.13923451 -1.13963133
   0.45299439]
 [ 0.55107783  0.54981237  0.54850537 ...  1.01623625  1.20255373
   1.95305622]
 ...
 [-0.88092053 -0.88167643 -0.88229015 ...  0.07907505 -0.10906991
  -0.67205199]
 [-0.7218096  -0.72262212 -0.72331287 ... -0.76437003  0.0783049
   0.64050212]
 [-1.04003146 -0.72262212 -0.56433559 ... -0.67065391 -0.85856912
  -0.95331358]]
[[13.]
 [ 4.]
 [13.]
 ...
 [ 7.]
 [15.]
 [53.]]
[[-0.40358774 -0.56356781 -0.56433559 ... -1.23295063 -0.48381951
  -1.23457517]
 [-1.35825331 -1.51789368 -1.83615384 ...  1.48481684  0.54674191
   0.73425598]
 [-0.56269867 -0.56356781 -0.40535831 ...  0.36022341 -0.67119432
  -0.76580585]
 ...
 [-0.7218096  -0.72262212 -0.72331287 ... -0.85808615 -1.23331873
  -1.23457517]
 [-2.09016358 -2.05867833 -2.01102885 ... -0.29578943 -0.48381951
  -0.76580585]
 [-0.88092053 -1.04073074 -1.04126744 ... -0.295789

# **Training**
![alt text](https://drive.google.com/uc?id=1xIXvqZ4EGgmxrp7c9r0LOVbcvd4d9H4N)
![alt text](https://drive.google.com/uc?id=1S42g06ON5oJlV2f9RukxawjbE4NpsaB6)
![alt text](https://drive.google.com/uc?id=1BbXu-oPB9EZBHDQ12YCkYqtyAIil3bGj)

(和上圖不同處: 下面的 code 採用 Root Mean Square Error)

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [14]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 2
iter_time = 10000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%5000==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)


0:50.16931699765628
5000:9.717033978795834


# **Testing**
![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

In [16]:
testdata = pd.read_csv('test.csv', header = None, encoding = 'big5')
#testdata = pd.read_csv('./hw1_Regression/DataSet/test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , 14.6089046 , 14.60777537, ..., -1.22275893,
        -1.59712273, -0.47423225],
       [ 1.        ,  9.29613618,  8.53645356, ..., -0.37996139,
        -0.66068927, -1.03642775],
       [ 1.        , 22.95754068, 22.19692764, ..., -1.87826813,
        -1.59712273, -1.13012699],
       ...,
       [ 1.        , 17.64477226, 18.4023515 , ..., -0.28631722,
        -0.75433261,  0.65015873],
       [ 1.        ,  7.01923543,  7.01862311, ..., -1.59733562,
        -1.69076608, -1.69232249],
       [ 1.        ,  9.29613618,  9.29536879, ...,  2.42936375,
         2.71047121,  1.21235422]])

# **Prediction**
說明圖同上

![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

有了 weight 和測試資料即可預測 target。

In [17]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[ 1.90652227e+02],
       [ 2.87394680e+02],
       [ 1.85205319e+02],
       [ 2.30983702e+02],
       [ 3.51515616e+02],
       [ 1.94720955e+02],
       [ 4.86918396e+02],
       [-2.12761747e+02],
       [ 5.73770823e+02],
       [ 8.73354971e+02],
       [ 5.92924980e+01],
       [ 5.31796694e+02],
       [ 1.46980909e+03],
       [-4.16286618e-01],
       [-2.78194386e+02],
       [ 3.50798543e+02],
       [ 8.10293740e+02],
       [ 1.13347664e+03],
       [ 8.05113828e+02],
       [ 3.84875935e+02],
       [ 4.34125624e+02],
       [-5.97547348e+02],
       [ 6.72128823e+02],
       [ 7.21351426e+02],
       [ 1.63616109e+02],
       [ 8.37382728e+01],
       [ 1.13822594e+02],
       [-3.04459109e+01],
       [ 6.95234483e+02],
       [ 1.99073282e+03],
       [ 6.45961184e+02],
       [ 4.47582812e+02],
       [ 4.60346552e+02],
       [ 2.32935057e+02],
       [ 1.32336757e+03],
       [ 1.27966182e+03],
       [ 3.52427266e+02],
       [ 1.93866255e+02],
       [ 1.0

# **Save Prediction to CSV File**


In [18]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 190.65222732653038]
['id_1', 287.39468007024647]
['id_2', 185.20531876728398]
['id_3', 230.98370187260798]
['id_4', 351.51561603837735]
['id_5', 194.7209552199874]
['id_6', 486.91839575848206]
['id_7', -212.76174685268566]
['id_8', 573.7708229658801]
['id_9', 873.3549706064462]
['id_10', 59.29249797993459]
['id_11', 531.796694324271]
['id_12', 1469.8090878332305]
['id_13', -0.41628661832247654]
['id_14', -278.19438594203314]
['id_15', 350.7985430792792]
['id_16', 810.293739960465]
['id_17', 1133.476638292685]
['id_18', 805.1138280188933]
['id_19', 384.8759354313405]
['id_20', 434.1256235818396]
['id_21', -597.5473484830891]
['id_22', 672.1288228090051]
['id_23', 721.3514256765254]
['id_24', 163.6161090117289]
['id_25', 83.7382727922862]
['id_26', 113.82259360238635]
['id_27', -30.445910915428186]
['id_28', 695.2344832964461]
['id_29', 1990.732816769095]
['id_30', 645.9611844980052]
['id_31', 447.5828121273297]
['id_32', 460.3465521448774]
['id_33', 232.93505701

相關 reference 可以參考:

Adagrad :
https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705 

RMSprop : 
https://www.youtube.com/watch?v=5Yt-obwvMHI 

Adam
https://www.youtube.com/watch?v=JXQT_vxqwIs 


以上 print 的部分主要是為了看一下資料和結果的呈現，拿掉也無妨。另外，在自己的 linux 系統，可以將檔案寫死的的部分換成 sys.argv 的使用 (可在 terminal 自行輸入檔案和檔案位置)。

最後，可以藉由調整 learning rate、iter_time (iteration 次數)、取用 features 的多寡(取幾個小時，取哪些特徵欄位)，甚至是不同的 model 來超越 baseline。

Report 的問題模板請參照 : https://docs.google.com/document/d/1s84RXs2AEgZr54WCK9IgZrfTF-6B1td-AlKR9oqYa4g/edit